## Инференциальная статистика

In [ ]:
# В этом разделе нужно определить, что с чем сравниваем
# Далее запускаем ячейки и получаем результат

df_for_test = df_clean  # датафрейм для анализа (очищенный от выбросов или исходный)

target = 'duration_total'      # целевая метрика, которую сравниваем (на пример: время выполнения поручения)
group = 'assignment_finished'  # группы для сравнения (можно разделить искусственно, задав условия в датафрейме и определить его в новое поле)

segments = ['division_name']   # разрезы (примеру, по дивизионам, можно взять несколько в список)

alpha = 0.05  # уровень значимости (5%)
bootstrap_n = 1000  # сколько раз пересэмплировать для bootstrap

In [ ]:
# Делим числовые и категориальные данные
num_cols = df_numeric.columns.tolist()
cat_cols = df_categorical.columns.tolist() + df_bool.columns.tolist()

print('Количество числовых колонок:', len(num_cols))
print('Количество категориальных колонок:', len(cat_cols))

Количество числовых колонок: 13
Количество категориальных колонок: 7


In [ ]:
# Выводим целевую метрику и группы для сравнения
print('Целевая метрика:', target)
print('Группы для сравнения:', group)

Целевая метрика: duration_total
Группы для сравнения: assignment_finished


In [ ]:
# ВЫБОР СТАТИСТИЧЕСКОГО ТЕСТА
def check_assumptions(df, target, group):
    # Разбиваем на группы
    groups = [g[target].dropna() for _, g in df.groupby(group)]
    if len(groups) < 2:
        return {"normal": False, "equal_var": False}
    
    # Шапиро тест (проверка нормального распределения)
    normal = True
    for g in groups:
        if len(g) > 3 and len(g) < 5000:
            if stats.shapiro(g)[1] < 0.05:
                normal = False

    # Проверка равенства дисперсий
    equal_var = stats.levene(*groups)[1] > 0.05
    return {"normal": normal, "equal_var": equal_var}

# Выбор теста с учетом распределения и разброса
def choose_test(df, target, group, a):
    n = df[group].nunique()

    if n == 2:
        return "t_test" if a["normal"] and a["equal_var"] else "mann_whitney"

    return "anova"

# Запуск теста
def run_test(df, target, group, test):
    groups = [g[target].dropna() for _, g in df.groupby(group)]

    if test == "t_test":
        stat, p = stats.ttest_ind(*groups)
    elif test == "mann_whitney":
        stat, p = stats.mannwhitneyu(*groups, alternative="two-sided")
    else:
        stat, p = stats.f_oneway(*groups)

    return stat, p

# Расчёт эффекта
def effect(df, target, group):
    groups = [g[target].dropna() for _, g in df.groupby(group)]
    if len(groups) < 2:
        return {"mean_diff": np.nan, "uplift": np.nan}

    diff = groups[1].mean() - groups[0].mean()
    return {"mean_diff": diff, "uplift": diff / groups[0].mean() * 100 if groups[0].mean() != 0 else 0}

# Bootstrap доверительный интервал
def bootstrap_ci(df, target, group, n=1000):
    groups = [g[target].dropna().values for _, g in df.groupby(group)]
    if len(groups) < 2:
        return np.nan, np.nan

    diffs = []
    for _ in range(n):
        a = np.random.choice(groups[0], len(groups[0]), True)
        b = np.random.choice(groups[1], len(groups[1]), True)
        diffs.append(b.mean() - a.mean())

    return np.percentile(diffs, [2.5, 97.5])

In [ ]:
a = check_assumptions(df_for_test, target, group)
test = choose_test(df_for_test, target, group, a)
stat, p = run_test(df_for_test, target, group, test)
eff = effect(df_for_test, target, group)
ci_low, ci_high = bootstrap_ci(df_for_test, target, group, bootstrap_n)

RESULT = {
    "target": target,
    "group": group,
    "test": test,
    "p_value": p,
    **a,
    **eff,
    "ci_low": ci_low,
    "ci_high": ci_high
}

pd.DataFrame([RESULT])

,target,group,test,p_value,normal,equal_var,mean_diff,uplift,ci_low,ci_high
0,duration_total,assignment_finished,mann_whitney,0.00,True,False,62.92,267.91,60.45,65.18


**Пояснение**
* test = какой использовался тест
* p_value = уровень значимости 
* normal = нормальность распределения
* equal_var = равенство дисперсий (одинаковый ли разброс в группах)
* mean_diff = разница средних значений
* uplift = относительный эффект (на сколько выросло в %)
* ci = стабильность (где, в каком интервале, может находиться эффект)

In [ ]:
def summary(r):
    sig = "Есть эффект ✅" if r["p_value"] < alpha else "Эффекта нет ❌"

    print(f'''
=== Итоговые результаты теста ===

Целевая метрика: {r['target']}
Группы: {r['group']}
Выбранный тест: {r['test']}

p-value: {r['p_value']:.4f} → {sig}
Абсолютный эффект (сколько добавилось): {r['mean_diff']:.3f}
Сколько добавилось в процентах: {r['uplift']:.2f}%

Реальная вероятно находится в интервале между: 
[{r['ci_low']:.3f}, {r['ci_high']:.3f}]
''')

summary(RESULT)


=== Итоговые результаты теста ===

Целевая метрика: duration_total
Группы: assignment_finished
Выбранный тест: mann_whitney

p-value: 0.0000 → Есть эффект ✅
Абсолютный эффект (сколько добавилось): 62.919
Сколько добавилось в процентах: 267.91%

Реальная вероятно находится в интервале между: 
[60.455, 65.176]



In [ ]:
SEGMENT_RESULTS = []

for seg in segments:
    for val, sdf in df_for_test.groupby(seg):
        try:
            a = check_assumptions(sdf, target, group)
            t = choose_test(sdf, target, group, a)
            stat, p = run_test(sdf, target, group, t)
            eff = effect(sdf, target, group)

            SEGMENT_RESULTS.append({
                "segment": seg,
                "value": val,
                "p": p,
                **eff
            })
        except:
            pass

print(f"{'\033[33m'}Результаты теста относительно сегментов (разрезов):{'\033[0m'}")
display(pd.DataFrame(SEGMENT_RESULTS))

Результаты теста относительно сегментов (разрезов):


,segment,value,p,mean_diff,uplift
0,division_name,03. див. Западная Сибирь,0.00,96.98,593.07
1,division_name,04. див. Урал,0.00,27.30,204.40
2,division_name,06. див. Средняя Волга,0.00,15.08,70.12
3,division_name,07. див. Верхняя Волга,0.00,34.95,86.07
4,division_name,08. див. Центральный,0.00,82.02,497.59
5,division_name,09. див. Черноземье,0.00,64.72,208.03
6,division_name,10. див. Приволжский,0.00,41.65,224.93
